# <u>Project 1 (Forecasting Excercise) </u>

There is a public API that you can use to download the data but they updated the website on the 18th of October and it no longer seems to work. 

To extract the data, I extract the names of the tickers for each column and put these into a csv. 

I then put these csvs into the GFD app and get an excel time series dataset in return.

---------------------------------------------------------------------------------------------------------------------------------------------


I was not sure what format to put the data into, so for many of them I have generated plots which are stored in corresponding files.

##  To access the percentage change in attribute for a certain ticker 

I used the original excel sheet as the basis for my storage.

In [1]:
# print(df_yearly_pct_changes[Column Name/Attribute].loc[FX_ticker])                 

# e.g. (df_yearly_pct_changes['Cash_rate'].loc['NLGUSD'])

##  To access the linear regression coefficients for a certain ticker


In [2]:
# print(df_coefficients[Column Name/Attribute].loc[FX_ticker])

# e.g. (df_coefficients['Cash_rate'].loc['NLGUSD'])

--------------------------

In [3]:
import scipy.stats as stats
import pandas as pd
import requests
import numpy as np
import csv
import os 
import sys
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import pickle
import re
import statsmodels.api as sm
from IPython.display import display, Image, HTML

import module as m  #this is a custom module


## Generate csvs of tickers

Using the columns in the database make individual csvs to plug into GFD

In [4]:
df_project_1 = pd.read_excel(r'Internship_Project_1.xlsx')  # Adjust this path as per your exact file location
fx_tickers = m.return_excel_tickers()

Saved column 'FX_ticker' to ./gfd_input_csvs\FX_ticker.csv
Saved column 'Contract_description' to ./gfd_input_csvs\Contract_description.csv
Saved column 'Traded_pair' to ./gfd_input_csvs\Traded_pair.csv
Saved column 'Cash_rate' to ./gfd_input_csvs\Cash_rate.csv
Saved column 'Real GDP' to ./gfd_input_csvs\Real_GDP.csv
Saved column 'Nominal GDP' to ./gfd_input_csvs\Nominal_GDP.csv
Saved column 'M0' to ./gfd_input_csvs\M0.csv
Saved column 'M1' to ./gfd_input_csvs\M1.csv
Saved column 'M3' to ./gfd_input_csvs\M3.csv
Saved column 'Industrial production' to ./gfd_input_csvs\Industrial_production.csv
Saved column 'CPI' to ./gfd_input_csvs\CPI.csv
Saved column 'PPI' to ./gfd_input_csvs\PPI.csv
Saved column 'FX Reserves' to ./gfd_input_csvs\FX_Reserves.csv
Saved column 'Unemployment rate' to ./gfd_input_csvs\Unemployment_rate.csv
Saved column 'P/E Ratio' to ./gfd_input_csvs\P_E_Ratio.csv
Saved column 'Dividend Yield' to ./gfd_input_csvs\Dividend_Yield.csv
Saved column 'Market Capitalization (USD

One problem with this is when you input the ticker symbols into GFD, some of them don't exist, but the inverse does.

## Fixing Ticker Symbols

Some of the FX_tickers do not exist but their inverse does, so I replace with their inverted version

In [5]:
filename = "gfd_input_csvs/Fx_ticker.csv"

os.makedirs(os.path.dirname(filename), exist_ok=True)

# List of unfound tickers
unfound_tickers = ['CLPUSD', 'CZKUSD', 'HUFUSD', 'INRUSD', 'MYRUSD', 'PHPUSD', 'RONUSD', 'THBUSD', 
                   'USDUSD', 'ATSUSD', 'BEFUSD', 'FRFUSD', 'ESPUSD', 'FIMUSD', 'GRDUSD', 'IEPUSD', 
                   'ISKUSD', 'ITLUSD', 'PTEUSD', 'NLGUSD', 'DEMUSD', 'RSDUSD']

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    for item in fx_tickers: #for every ticker in the main FX_ticker list
        if item != 'USDUSD':  # Exclude 'USDUSD' since this ticker does not exist
            if item in unfound_tickers:  #if this ticker cannot be found in the normal format, flip it
                writer.writerow([item[3:] + item[:3]])  # Flip the ticker (e.g., USD -> EURUSD becomes USDEUR)
            else:
                writer.writerow([item])


I also want to generate a csv of all possible tickers as these can potentially be used to extend the ticker time series data.

In [6]:
inverted_ticker_list = []
filename = "gfd_input_csvs/Fx_ticker_inverted.csv" #inverted ticker csv filepath

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    for item in fx_tickers:
        if item != 'USDUSD':  # Exclude 'USDUSD' since this ticker does not exist
            if item not in unfound_tickers :
                writer.writerow([item[3:] + item[:3]])  # Flip the ticker (e.g., USD -> EURUSD becomes EURUSD)

## Fixing some of the tickers

Some of the ticker symbols when I tried to put into GFD did not work so I have removed them from their ticker lists

In [7]:
m.filter_and_save_csv('gfd_input_csvs/Nominal_GDP.csv', 'GDPCHF')
m.filter_and_save_csv('gfd_input_csvs/Real_GDP.csv', 'GDPCCHF')
m.filter_and_save_csv('gfd_input_csvs/TBill3M.csv', 'ITROU3D')

--------
## Creating Time Series Database

In the excel sheets folder, there is an excel sheet for each of the attributes which stores the time series data which is pulled from the GFD database.

All the time series data is stored in sheets called Price, Price 1 or Price 2 or Price 1, Price 2 & Price 3



## Create Dictionary of Time Series Data

In [8]:
excel_sheets_folder = 'excel_sheets'   # Path to the folder with Excel files

ticker_dict = m.process_excel_sheets(excel_sheets_folder, df_project_1, unfound_tickers) #convert into {'ticker': time_series_df} format
ticker_dict = m.extend_main_with_inverted(excel_sheets_folder, ticker_dict) #extend time series data where possible using the inverted ticker

with open('ticker_dict.pkl', 'wb') as f: #The notebook can sometimes overwrite variables so I used pickle to store and load variables
    pickle.dump(ticker_dict, f) 

This creates a large database dictionary of all the timeseries data scraped from GFD.

It takes a while to run [10 mins ish]


In [9]:
print(ticker_dict['ITAUS3D'])

             Ticker  Open  High  Low   Close  Volume
Date                                                
1928-07-31  ITAUS3D   NaN   NaN  NaN  4.5000     NaN
1928-08-31  ITAUS3D   NaN   NaN  NaN  4.5000     NaN
1928-09-30  ITAUS3D   NaN   NaN  NaN  4.5000     NaN
1928-10-31  ITAUS3D   NaN   NaN  NaN  4.5000     NaN
1928-11-30  ITAUS3D   NaN   NaN  NaN  4.5000     NaN
...             ...   ...   ...  ...     ...     ...
2024-10-18  ITAUS3D   NaN   NaN  NaN  4.3970     NaN
2024-10-21  ITAUS3D   NaN   NaN  NaN  4.3700     NaN
2024-10-22  ITAUS3D   NaN   NaN  NaN  4.4060     NaN
2024-10-23  ITAUS3D   NaN   NaN  NaN  4.3976     NaN
2024-10-24  ITAUS3D   NaN   NaN  NaN  4.3837     NaN

[10575 rows x 6 columns]


## part a) Generate JPGs

Within the project 1 folder, there is a 'jpg_time_series_ticker' folder, this contain figures for all the FX_tickers

In [10]:
valid_tickers = df_project_1['FX_ticker'].tolist()  # Load valid tickers from a DataFrame column
folder_name = 'jpg_time_series_ticker' #folder name to store jpg

with open('ticker_dict.pkl', 'rb') as f:
    ticker_dict = pickle.load(f)  # Load the dictionary with time series data

# Generate and save the plots
m.plot_and_save_ticker_data(ticker_dict, valid_tickers, folder_name)


## part b) Finding outliers 

To identify outliers in the data, there are numerous approaches; however, given the extensive time span of the data (e.g., spanning 300 years), I have opted to use the yearly percentage change in price.

Running the code above generates plots organized within folders named after each ticker symbol. Inside each folder, there are two plots: one representing the raw price data and the other showing the percentage change.

To pinpoint outliers within each dataset, I calculate the largest percentage price change by magnitude and display this value as text on the corresponding image as shown in the example below.

In [11]:
image_path1 = 'Figures\\SGDUSD_time_series.jpg'
image_path2 = 'Figures\\SGDUSD_yearly_pct_change.jpg'

# Display images side by side
display(HTML(f"""
<div style="display: flex; align-items: center;">
    <div><img src="{image_path1}" style="width: 80%; margin-right: 1%;"></div>
    <div><img src="{image_path2}" style="width: 80%;"></div>
</div>
"""))

## Create a DataFrame of DataFrames 

To store time series data in a similar format to original excel.

In [12]:
ticker_dict_file = 'ticker_dict.pkl'
excel_file = 'Internship_Project_1.xlsx'

df_time_series_filled = m.load_and_fill_time_series(ticker_dict_file, excel_file)

## part e) Percentage change for all variables

Create indentical dataframe that calculates the percentage change for all the time series data.

In [13]:
df_yearly_pct_changes = m.calculate_yearly_pct_changes(df_project_1, df_time_series_filled)  #Dataframe creation for yearly percentage changes

Example of how to extract percentage change any variable

In [14]:
print(df_yearly_pct_changes['Nominal GDP'].loc['BRLUSD'])

            Yearly % Change
Date                       
1861-01-01              NaN
1862-01-01              NaN
1863-01-01        11.111111
1864-01-01         3.333333
1865-01-01       -19.354839
...                     ...
2020-01-01         5.893342
2021-01-01         7.219311
2022-01-01        14.684334
2023-01-01        12.192136
2024-01-01         8.454360

[164 rows x 1 columns]


## part f) Linear Regression of percentage change onto percentage change 

In [15]:
df_coefficients = df_yearly_pct_changes.copy()  # To hold time series of coefficients (intercept and slope)
df_coefficients = m.calculate_coefficients_over_time(df_yearly_pct_changes, start_year=1984)  # linear regression coefficient for percentage change with expanding window




This generates a similar dataframe structure but this time it is the linear regression coefficients for each year onwards from 1984.

It looks like this for example....

In [16]:
print((df_yearly_pct_changes['TBill3M'].loc['CLPUSD']))  #Example Use
print((df_coefficients['TBill3M'].loc['CLPUSD']))   #Example Use

            Yearly % Change
Date                       
1997-01-01              NaN
1998-01-01              NaN
1999-01-01        26.904986
2000-01-01       -17.790956
2001-01-01       -16.140667
2002-01-01       -30.000000
2003-01-01       -55.760369
2004-01-01       -10.416667
2005-01-01       -13.178295
2006-01-01       112.053571
2007-01-01         7.368421
2008-01-01        20.588235
2009-01-01        33.008130
2010-01-01       -94.132029
2011-01-01       608.333333
2012-01-01        50.294118
      Intercept     Slope
Year                     
2000  -9.343094  0.072236
2001  -8.487249  0.042402
2002  -8.995162  0.081062
2003  -8.954740  0.025896
2004  -2.860474  0.083005
2005  -1.167687  0.093071
2006  -1.321439  0.085639
2007  -1.621287  0.084595
2008  -0.957893  0.091260
2009  -2.950887  0.057037
2010  -0.471004 -0.051088
2011  -0.047941  0.009321
2012  -0.779164  0.009157
2013  -0.779164  0.009157
2014  -0.779164  0.009157
2015  -0.779164  0.009157
2016  -0.779164  0.009157
20

## part g/h) Forecast using linear regression coefficients

The outputs are saved in graphs in their corresponding folders with the DM and p value as part of the legend.

See example jpg below.

In [17]:
folder_name = 'jpg_time_series_ticker'
m.process_ticker_data(df_coefficients, df_yearly_pct_changes, folder_name)        

Saved forecast plot for AUDUSD - Cash_rate at jpg_time_series_ticker\AUDUSD\Cash_rate_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - Real GDP at jpg_time_series_ticker\AUDUSD\Real_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - Nominal GDP at jpg_time_series_ticker\AUDUSD\Nominal_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - M0 at jpg_time_series_ticker\AUDUSD\M0_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - M1 at jpg_time_series_ticker\AUDUSD\M1_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - M3 at jpg_time_series_ticker\AUDUSD\M3_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - CPI at jpg_time_series_ticker\AUDUSD\CPI_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - PPI at jpg_time_series_ticker\AUDUSD\PPI_forecasted_exchange_rate.jpg
Saved forecast plot for AUDUSD - FX Reserves at jpg_time_series_ticker\AUDUSD\FX_Reserves_forecasted_exchange_rate.jpg
Saved forecast plot for A

c:\Finance Internships\QRT Technical\Project 1\module\forecasting.py:94: RuntimeWarning: invalid value encountered in subtract
  forecast_errors_model = forecasted_exchange_rates - forecasted_exchange_rates  # Model forecast errors (difference from itself, likely a placeholder)


Saved forecast plot for NOKUSD - M0 at jpg_time_series_ticker\NOKUSD\M0_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - M1 at jpg_time_series_ticker\NOKUSD\M1_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - M3 at jpg_time_series_ticker\NOKUSD\M3_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - Industrial production at jpg_time_series_ticker\NOKUSD\Industrial_production_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - CPI at jpg_time_series_ticker\NOKUSD\CPI_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - PPI at jpg_time_series_ticker\NOKUSD\PPI_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - FX Reserves at jpg_time_series_ticker\NOKUSD\FX_Reserves_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - Unemployment rate at jpg_time_series_ticker\NOKUSD\Unemployment_rate_forecasted_exchange_rate.jpg
Saved forecast plot for NOKUSD - P/E Ratio at jpg_time_series_ticker\NOKUSD\P_E_Ratio_forecasted_excha

c:\Finance Internships\QRT Technical\Project 1\module\forecasting.py:94: RuntimeWarning: invalid value encountered in subtract
  forecast_errors_model = forecasted_exchange_rates - forecasted_exchange_rates  # Model forecast errors (difference from itself, likely a placeholder)


Saved forecast plot for SEKUSD - Real GDP at jpg_time_series_ticker\SEKUSD\Real_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - Nominal GDP at jpg_time_series_ticker\SEKUSD\Nominal_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - M0 at jpg_time_series_ticker\SEKUSD\M0_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - Industrial production at jpg_time_series_ticker\SEKUSD\Industrial_production_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - CPI at jpg_time_series_ticker\SEKUSD\CPI_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - PPI at jpg_time_series_ticker\SEKUSD\PPI_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - FX Reserves at jpg_time_series_ticker\SEKUSD\FX_Reserves_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - Unemployment rate at jpg_time_series_ticker\SEKUSD\Unemployment_rate_forecasted_exchange_rate.jpg
Saved forecast plot for SEKUSD - P/E Ratio at jpg_time_series_ticker\SEK

c:\Finance Internships\QRT Technical\Project 1\module\forecasting.py:94: RuntimeWarning: invalid value encountered in subtract
  forecast_errors_model = forecasted_exchange_rates - forecasted_exchange_rates  # Model forecast errors (difference from itself, likely a placeholder)


Saved forecast plot for DKKUSD - Cash_rate at jpg_time_series_ticker\DKKUSD\Cash_rate_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - Real GDP at jpg_time_series_ticker\DKKUSD\Real_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - Nominal GDP at jpg_time_series_ticker\DKKUSD\Nominal_GDP_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - M0 at jpg_time_series_ticker\DKKUSD\M0_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - M1 at jpg_time_series_ticker\DKKUSD\M1_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - M3 at jpg_time_series_ticker\DKKUSD\M3_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - Industrial production at jpg_time_series_ticker\DKKUSD\Industrial_production_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - CPI at jpg_time_series_ticker\DKKUSD\CPI_forecasted_exchange_rate.jpg
Saved forecast plot for DKKUSD - PPI at jpg_time_series_ticker\DKKUSD\PPI_forecasted_exchange_rate.jpg
Saved

In [18]:
image_path1 = 'Figures\\Nominal_GDP_forecasted_exchange_rate.jpg'
image_path2 = 'Figures\\TBond10Y_forecasted_exchange_rate.jpg'

# Display images side by side
display(HTML(f"""
<div style="display: flex; align-items: center;">
    <div><img src="{image_path1}" style="width: 80%; margin-right: 1%;"></div>
    <div><img src="{image_path2}" style="width: 80%;"></div>
</div>
"""))